In [1]:
import os, sys, time
import rasterio

import networkx as nx
import geopandas as gpd
import pandas as pd

In [2]:
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append("../")
import GOSTnets as gn

In [3]:
pth = "./" # change this path to your working folder
G = nx.read_gpickle(os.path.join(pth, 'tutorial_outputs', r'iceland_network_clean.pickle'))

In [4]:
G_time = gn.convert_network_to_time(G, distance_tag = 'length', road_col = 'infra_type', factor = 1000)

# Gravity calculations

When we discuss gravity calculations, we are combining two measures to identify the attractiveness between pairs of locations
1. Travel time between origins and destinations
2. Weight of the value of interest at both the origin and destination

‘Market access’ is a measure of accessibility from one origin to all destinations, based purely on time. The outcome of this analysis is often visualized as an ‘isochrone’, centered on a given origin point. Isochrones depict how far away from the origin you can move, in all directions, within a certain time frame. (E.g. 1 hour).

‘Market potential’ can be assessed by weighting all potential destinations by a factor designed to represent their attractiveness to the origin point. In the context of performing commercial site selection, such factors might include GDP per capita or population (positively related), or the number of other firms (negatively related). We refer to each origin – destination pair as an ‘O-D’ pair, and the matrix of travel times from all origin points to all destination points as an ‘O-D matrix’. An example of such a matrix is given below for five origin and destination points, A to E:

In [5]:
# Sample rek_grid with population
rek_grid_file = os.path.join(pth, "tutorial_data", "rek_grid.shp")
rek_pop_grid_file = rek_grid_file.replace(".shp", "_pop.shp")
if not os.path.exists(rek_pop_grid_file):
    population_data = "R:\GLOBAL\POP&DEMO\LandScan_2012\lspop2012.tif"

    in_grid = gpd.read_file(rek_grid_file)    
    in_pop = rasterio.open(population_data)
    in_grid = in_grid.to_crs(in_pop.crs)

    geoms = ([x.x, x.y] for x in in_grid['geometry'])
    vals = in_pop.sample(geoms)
    in_grid['Pop'] = [x[0]+1 for x in vals]
    
    in_grid.to_file(rek_pop_grid_file)

In [6]:
rek_grid = gpd.read_file(rek_pop_grid_file)
churches_file = os.path.join(pth, "tutorial_data", "churches.shp")
in_churches = gpd.read_file(churches_file)
in_churches = in_churches.to_crs(rek_grid.crs)

In [7]:
origins_df = gn.pandana_snap_c(G_time, rek_grid, source_crs='epsg:4326', target_crs='epsg:4326',add_dist_to_node_col = True)
origins = list(set(origins_df['NN']))
destinations_df = gn.pandana_snap_c(G_time, in_churches, source_crs='epsg:4326', target_crs='epsg:4326',add_dist_to_node_col = True)
destinations = list(set(destinations_df['NN']))

In [8]:
OD = gn.calculate_OD(G_time, origins, destinations, fail_value = 9999999)

In [10]:
import GOSTnets.calculate_od_raw as calcOD
gravity = calcOD.calculate_gravity(OD, oWeight=[1] * OD.shape[0], dWeight = destinations_df['Pop'])
gravity['NN'] = origins

In [19]:
pd.merge?

In [21]:
origins_gravity = pd.merge(origins_df, gravity, on="NN")
origins_gravity.to_file(rek_pop_grid_file.replace(".shp", "_gravity.shp"))